<a href="https://colab.research.google.com/github/trefftzc/cis677/blob/main/Voronoi_c_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now a version that runs on the GPU.
It is also compiled with openmp.

In [3]:
%%writefile voronoi_openmp_gpu.c
//
// compile with "gcc voronoi_openmp_gpu.c -o voronoi_openmp_gpu -lm -fopenmp"
// run with ./voronoi_openmp"

#include <math.h>
#include <omp.h>
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

#define SEED_ROW_SIZE 4
#define SEED_COL_SIZE 2

void calculate_voronoi(int, double *, double *);
int gen_voronoi_graph(int, double *);

int main(int argc, char *argv[]) {
  int size = 4096;
  double(*area)[size] = calloc(size, sizeof *area);

  double seed_vals[SEED_ROW_SIZE][SEED_COL_SIZE] = {
      {0, 0}, {0, size - 1}, {size - 1, size - 1}, {size - 1, 0}};

  double *seeds = (double *)malloc(sizeof(seed_vals));
  memcpy(seeds, seed_vals, sizeof(seed_vals));

  double start = omp_get_wtime();
  calculate_voronoi(size, (double *)seeds, (double *)area);
  double end = omp_get_wtime();

  printf("PARALLEL VORONOI GPU IMPLEMENTATION\n");
  printf("-------------------------------\n");
  printf("Array size: %d\n", size);
  printf("Time taken: %f s\n", end - start);

  free(seeds);
  free(area);


}

void calculate_voronoi(int size, double *pSeeds, double *pArea) {
  double closest_seed, closest_distance, dist;

  #pragma omp target
  #pragma omp loop
  for (int i = 0; i < size; i++) {
    for (int j = 0; j < size; j++) {
      closest_seed = -1;
      closest_distance = pow(size, 2);

      for (int k = 0; k < SEED_ROW_SIZE; k++) {
        dist = sqrt(pow(pSeeds[k * SEED_COL_SIZE + 0] - i, 2) +
                    pow(pSeeds[k * SEED_COL_SIZE + 1] - j, 2));

        if (dist < closest_distance) {
          closest_distance = dist;
          closest_seed = k;
        }
      }
      pArea[i * size + j] = closest_seed;
    }
  }
}


Writing voronoi_openmp_gpu.c


Now compile the code:

In [4]:
!gcc voronoi_openmp_gpu.c -o voronoi_openmp_gpu -lm -fopenmp -O3

And now time and execute:

In [5]:
!time ./voronoi_openmp_gpu

PARALLEL VORONOI GPU IMPLEMENTATION
-------------------------------
Array size: 4096
Time taken: 0.195627 s

real	0m0.205s
user	0m0.130s
sys	0m0.075s
